In [17]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationKGMemory(llm=llm,return_messages=True)

def add_message(input, output):
    memory.save_context({"input": input}, {'output': output})

add_message("Hi i'm Nicolas, I live in South Korea", "Wow that is so cool!")



In [18]:
memory.load_memory_variables({'input': "who is Nicolas"})

{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea.')]}

In [19]:
add_message("Nicolas likes kimchi", "Wow that is so cool!")

In [20]:
memory.load_memory_variables({'input': "what does nicolas like"})

{'history': [SystemMessage(content='On Nicolas: Nicolas is a person. Nicolas lives in South Korea. Nicolas likes kimchi.')]}